# **ResNet 구현**

### **라이브러리 호출**

In [ ]:
!pip uninstall -y sympy
!pip install sympy

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 43.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import time

In [ ]:
import cv2
from torch.utils.data import DataLoader, Dataset
from PIL import Image

### **이미지 데이터 전처리**

In [ ]:
class ImageTransform():
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
      }

  def __call__(self, img, phase):
    return self.data_transform[phase](img)

### **변수 값 정의**

In [ ]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32

### **데이터 셋 로딩**

In [ ]:
cat_directory = r'../content/drive/MyDrive/Euron-Data/chap06/dogs-vs-cats/Cat/'
dog_directory = r'../content/drive/MyDrive/Euron-Data/chap06/dogs-vs-cats/Dog/'

cat_images_filepaths = sorted([os.path.join(cat_directory, f) for f in os.listdir(cat_directory)])
dog_images_filepaths = sorted([os.path.join(dog_directory, f) for f in os.listdir(dog_directory)])
images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]

### **데이터 셋 분리; train, validation, test**

In [ ]:
random.seed(42)
random.shuffle(correct_images_filepaths)
train_images_filepaths = correct_images_filepaths[:400]
val_imgaes_filepaths = correct_images_filepaths[400:-10]
test_images_filepaths = correct_images_filepaths[-10:]
print(len(train_images_filepaths),len(val_imgaes_filepaths),len(test_images_filepaths))

400 92 10


### **이미지 label 구분 및 데이터셋 정의**

In [ ]:
class DogvsCatDataset(Dataset):
  def __init__(self, file_list, transform=None, phase='train'):
    self.file_list = file_list
    self.transform = transform
    self.phase = phase

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, idx):
    img_path = self.file_list[idx]
    img = Image.open(img_path)
    img_transformed = self.transform(img, self.phase)

    label = img_path.split('/')[-1].split('.')[0]
    if label == 'dog':
      label = 1
    elif label == 'cat':
      label = 0
    return img_transformed, label

In [ ]:
train_dataset = DogvsCatDataset(train_images_filepaths, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = DogvsCatDataset(val_imgaes_filepaths, transform=ImageTransform(size, mean, std), phase='val')

index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
0


### **데이터 불러오기**

In [ ]:
train_iterator = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_iterator = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloader_dict = {'train': train_iterator, 'val': valid_iterator}

batch_iterator = iter(train_iterator)
inputs, label = next(batch_iterator)
print(inputs.size())
print(label)

torch.Size([32, 3, 224, 224])
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 0, 1, 1, 0, 1, 0])


### **기본 블록 정의**

In [ ]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_channels, out_channels, stride=1, downsample=False):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                            stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                            stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)

    if downsample:
      conv = nn. Conv2d(in_channels, out_channels, kernel_size=1,
                        stride=stride, bias=False)
      bn = nn.BatchNorm2d(out_channels)
      downsample = nn. Sequential (conv, bn)
    else:
      downsample = None
    self.downsample = downsample

  def forward(self,x):
    i = x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)

    if self.downsample is not None:
      i = self.downsample(i)

    x += i
    x = self.relu(x)

    return x

### **병목 블록 정의**

In [ ]:
class Bottleneck(nn.Module):
  expansion = 4

  def __init__ (self, in_channels, out_channels, stride=1, downsample=False):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                           stride=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                           stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.conv3 = nn.Conv2d(out_channels, self.expansion*out_channels, kernel_size=1,
                           stride=1, bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion*out_channels)
    self.relu = nn.ReLU(inplace=True)

    if downsample:
      conv = nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1,
                       stride=stride, bias=False)
      bn = nn.BatchNorm2d(self.expansion*out_channels)
      downsample = nn.Sequential(conv, bn)
    else:
      downsample = None
    self.downsample = downsample

  def forward(self, x):
    i = x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bn3(x)

    if self.downsample is not None:
      i = self.downsample(i)

    x += i
    x = self.relu(x)

    return x

### **ResNet 네트워크**

In [ ]:
#from importlib import invalidate_caches
class ResNet(nn.Module) :
  def __init__(self, config, output_dim, zero_init_residual=False):
    super().__init__()

    block, n_blocks, channels = config
    self.in_channels = channels[0]
    assert len(n_blocks) == len(channels) == 4

    self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2,
                           padding=3, bias=False)
    self.bn1 = nn.BatchNorm2d(self.in_channels)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.layer1 = self.get_resnet_layer(block, n_blocks[0], channels[0])
    self.layer2 = self.get_resnet_layer(block, n_blocks[1], channels[1], stride=2)
    self.layer3 = self.get_resnet_layer(block, n_blocks[2], channels[2], stride=2)
    self.layer4 = self.get_resnet_layer(block, n_blocks[3], channels[3], stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(self.in_channels, output_dim)

    if zero_init_residual:
      for m in self.modules():
        if isinstance(m, Bottleneck):
          nn.init.constant_(m.bn3.weight, 0)
        elif isinstance(m, BasicBlock):
          nn.init.constant_(m.bn2.weight, 0)

  def get_resnet_layer(self, block, n_blocks, channels, stride=1):
    layers=[]
    if self.in_channels != block.expansion*channels:
      downsample = True
    else:
      downsample = False

    layers.append(block(self.in_channels, channels, stride, downsample))
    for i in range(1, n_blocks):
      layers.append(block(block.expansion*channels, channels))

    self.in_channels = block.expansion*channels
    return nn.Sequential(*layers)

  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    h = x.view(x.shape[0],-1)
    x = self.fc(h)

    return x, h

### **ResNetConfig 정의**

In [ ]:
ResNetConfig = namedtuple('ResNetConfig', ['block', 'n_blocks', 'channels'])

#기본 블록 사용
resnet18_config = ResNetConfig(block=BasicBlock, n_blocks=[2,2,2,2], channels=[64,128,256,512])
resnet34_config = ResNetConfig(block=BasicBlock, n_blocks=[3,4,6,3],channels=[64,128,256,512])

#병목 블록 사용
resnet50_config = ResNetConfig(block=Bottleneck, n_blocks=[3,4,6,3], channels=[64,128,256,512])
resnet101_config = ResNetConfig(block=Bottleneck, n_blocks=[3,4,23,3], channels=[64,128,256,512])
resnet152_config = ResNetConfig(block=Bottleneck, n_blocks=[3,8,36,3], channels=[64, 128,256,512])

### **Pretrained 모델 가져오기**

In [ ]:
pretrained_model= models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.6MB/s]


In [ ]:
print(pretrained_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### **정의한 ResNet 사용**

In [ ]:
OUTPUT_DIM = 2
model = ResNet(resnet50_config, OUTPUT_DIM)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### **옵티마이저&손실 함수 정의**

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=1e-7)
criterion = nn.CrossEntropyLoss()

### **정확도 측정 함수 정의**

In [ ]:
def calculate_topk_accuracy(y_pred, y, k=2):
  with torch.no_grad():
    batch_size = y.shape[0]
    _, top_pred = y_pred.topk(k,1)
    top_pred = top_pred.t()
    correct = top_pred.eq(y.view(1,-1).expand_as(top_pred))
    correct_1 = correct[:1].reshape(-1).float().sum(0, keepdim=True)
    correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
    acc_1 = correct_1/batch_size
    acc_k = correct_k/batch_size
  return acc_1, acc_k

### **모델 학습/평가/시간 측정 함수 정의**

In [ ]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc_1 = 0
  epoch_acc_5 = 0
  model.train()
  for(x,y) in iterator:
    optimizer.zero_grad()
    y_pred=model(x)
    loss = criterion(y_pred[0],y)

    acc_1, acc_5 = calculate_topk_accuracy(y_pred[0], y)
    loss. backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc_1 += acc_1.item()
    epoch_acc_5 += acc_5.item()

  epoch_loss /= len(iterator)
  epoch_acc_1 /= len(iterator)
  epoch_acc_5 /= len(iterator)
  return epoch_loss, epoch_acc_1, epoch_acc_5

In [ ]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc_1 = 0
  epoch_acc_5 = 0
  model. eval()

  with torch.no_grad():
    for (x, y) in iterator:
      y_pred = model (x)
      loss = criterion (y_pred[0], y)

      acc_1, acc_5 = calculate_topk_accuracy (y_pred [0], y)
      epoch_loss += loss.item()
      epoch_acc_1 += acc_1.item()
      epoch_acc_5 += acc_5.item()

    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time/60)
  elapsed_secs = int(elapsed_time-(elapsed_mins*60))
  return elapsed_mins, elapsed_secs

### **모델 학습**

In [ ]:
best_valid_loss = float('inf')
EPOCHS = 10

path = "../content/drive/MyDrive/Euron-Data/chap06/ResNet-model.pt"

for epoch in range(EPOCHS):
  start_time = time.monotonic()
  train_loss, train_acc_1, train_acc_5 = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc_1, valid_acc_5 = evaluate(model, valid_iterator, criterion)
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), path)

  end_time = time.monotonic()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc @1: {train_acc_1*100:6.2f}% | ' \
        f'Train Acc @5: {train_acc_5*100:6.2f}%')
  print(f'\tValid Loss: {valid_loss:.3f} | Valid Ac @1: {valid_acc_1*100:6.2f}% | ' \
        f'Valid Acc @5: {valid_acc_5*100:6.2f}%')

Epoch: 01 | Epoch Time: 5m 31s
	Train Loss: 0.687 | Train Acc @1:  53.85% | Train Acc @5: 100.00%
	Valid Loss: 0.689 | Valid Ac @1:  48.21% | Valid Acc @5: 100.00%
Epoch: 02 | Epoch Time: 5m 26s
	Train Loss: 0.684 | Train Acc @1:  53.85% | Train Acc @5: 100.00%
	Valid Loss: 0.689 | Valid Ac @1:  52.53% | Valid Acc @5: 100.00%
Epoch: 03 | Epoch Time: 5m 23s
	Train Loss: 0.693 | Train Acc @1:  50.00% | Train Acc @5: 100.00%
	Valid Loss: 0.691 | Valid Ac @1:  50.30% | Valid Acc @5: 100.00%
Epoch: 04 | Epoch Time: 5m 21s
	Train Loss: 0.689 | Train Acc @1:  55.53% | Train Acc @5: 100.00%
	Valid Loss: 0.687 | Valid Ac @1:  49.11% | Valid Acc @5: 100.00%
Epoch: 05 | Epoch Time: 5m 24s
	Train Loss: 0.686 | Train Acc @1:  52.64% | Train Acc @5: 100.00%
	Valid Loss: 0.686 | Valid Ac @1:  50.30% | Valid Acc @5: 100.00%
Epoch: 06 | Epoch Time: 5m 34s
	Train Loss: 0.683 | Train Acc @1:  54.09% | Train Acc @5: 100.00%
	Valid Loss: 0.681 | Valid Ac @1:  50.45% | Valid Acc @5: 100.00%
Epoch: 07 | Epoc

In [ ]:
import pandas as pd
path = "../content/drive/MyDrive/Euron-Data/chap06/ResNet.csv"
id_list = []
pred_list = []
_id = 0
with torch.no_grad():
  for test_path in test_images_filepaths:
    img = Image.open(test_path)
    _id = test_path.split('/')[-1].split('.')[1]
    transform = ImageTransform(size, mean, std)
    img = transform(img, phase='val')
    img = img.unsqueeze(0)

    model.eval()
    outputs = model(img)
    preds = F.softmax(outputs[0], dim=1)[:, 1]. tolist()
    id_list.append(_id)
    pred_list.append(preds[0])

res = pd.DataFrame({
  'id' : id_list,
  'label' : pred_list
})
res.sort_values(by='id', inplace=True)
res.reset_index(drop=True, inplace=True)
res.to_csv(path, index=False)
res.head(10)

In [ ]:
class_ = classes = {0:'cat', 1:'dog'}
def display_image_grid(images_filepaths, predicted_labels=(), cols=5):
  rows = len(images_filepaths) // cols
  figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
  for i, image_filepath in enumerate(images_filepaths):
    image = cv2.imread (image_filepath)
    image = cv2.cvtColor (image, cv2. COLOR_BGR2RGB)
    a = random.choice(res['id'].values)
    label = res.loc[res['id'] == a, 'label'].values[0]
    if label > 0.5:
      label = 1
    else:
      label = 0
    ax. ravel()[i]. imshow(image)
    ax.ravel()[i].set_title(class_[label])
    ax.ravel()(i).set_axis_off()
  plt. tight_layout()
  plt.show()
display_image_grid(test_images_filepaths)